In [ ]:
%matplotlib inline
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable   
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path

module_path = Path().joinpath('../..')
if module_path not in sys.path:
    sys.path.append(module_path)
    
from collections import namedtuple
import multifidelityfunctions as mff
import multiLevelCoSurrogates as mlcs

np.random.seed(20160501)  # Setting seed for reproducibility

from pprint import pprint
np.set_printoptions(linewidth=200, edgeitems=10, precision=4, suppress=True)
plot_dir = Path('../../plots/')
data_dir = Path('../../files/')

from function_defs import *

In [ ]:
sample = np.load(f'{data_dir}2d_test_sample.npy')
plt.scatter(sample[:,0], sample[:,1])
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_sample_distribution.pdf')
plt.show()

# Error distribution

In [ ]:
aggregation = ['mean', 'median'][1]
lin = ['', 'lin_'][1]

errors = np.load(f'{data_dir}2d_{lin}error_tracking.npy')

aggregate = {
    'mean': np.mean,
    'median': np.median,
}

mean_errors = aggregate[aggregation](errors, axis=(0,1,2))

In [ ]:
norm = colors.LogNorm(vmin=np.min(mean_errors[0]), vmax=np.max(mean_errors[0]))
img = plt.scatter(sample[:,0], sample[:,1], c=mean_errors[0], cmap='viridis_r', norm=norm)
plt.title(f'{aggregation.title()} error - high fidelity (hierarchical) model')
plt.colorbar(img, orientation='vertical')
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_{lin}sample_{aggregation}_error_hierarchical.pdf')
plt.show()

In [ ]:
norm = colors.LogNorm(vmin=np.min(mean_errors[1]), vmax=np.max(mean_errors[1]))
img = plt.scatter(sample[:,0], sample[:,1], c=mean_errors[1], cmap='viridis_r', norm=norm)
plt.title(f'{aggregation.title()} error - high fidelity (direct) model')
plt.colorbar(img, orientation='vertical')
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_{lin}sample_{aggregation}_error_high.pdf')
plt.show()

In [ ]:
norm = colors.LogNorm(vmin=np.min(mean_errors[2]), vmax=np.max(mean_errors[2]))
img = plt.scatter(sample[:,0], sample[:,1], c=mean_errors[2], cmap='viridis_r', norm=norm)
plt.title(f'{aggregation.title()} error - low fidelity (direct) model')
plt.colorbar(img, orientation='vertical')
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_{lin}sample_{aggregation}_error_low.pdf')
plt.show()

In [ ]:
bins = [b*10**e for e in range(-2, 14, 1) for b in range(1, 11, 1)]
for i in range(3):
    plt.hist(errors[:,:,:,i,:].flatten(), label=i, bins=bins, cumulative=-1, log=True, alpha=.5)
# plt.yscale('log')
plt.xscale('log')
plt.legend(loc=0)
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_{lin}sample_error_hist.pdf')
plt.show()

# MSE errors per sample size combination

In [ ]:
sorted(data_dir.iterdir())

## Random sampling

In [ ]:
if '2d_mse_tracking.npy' in data_dir.iterdir:
    mse_tracking = np.load(f'{data_dir}2d_mse_tracking.npy')
else:
    mse_tracking = create_mse_tracking(TD_inv, low_random_sample)
    np.save(f'{data_dir}2d_mse_tracking.npy', mse_tracking)

In [ ]:
print('median')
pprint([(f'{95+i}%-ile', np.nanpercentile(np.nanmedian(mse_tracking, axis=2).flatten(), 95+i)) for i in range(6)])

In [ ]:
name = '2d-high-low-samples-random'
plot_high_vs_low_num_samples(mse_tracking, name, vmax=1000, save_as=f'{plot_dir}{name}.png')

In [ ]:
name = '2d-high-low-samples-random'
plot_high_vs_low_num_samples_diff(mse_tracking, name, vmax=10000, save_as=f'{plot_dir}{name}_diff.pdf')

## LHS

In [ ]:
def display_paired_differences(data, vmax=5, num_colors=5, save_as=None):
    paired_differences = data[:,:,:,0] - data[:,:,:,1]
    mean_paired_diff = np.mean(paired_differences, axis=2)
    std_paired_diff = np.std(paired_differences, axis=2, ddof=1)
    se_paired_diff = std_paired_diff / np.sqrt(data.shape[2])
    t_scores = abs(mean_paired_diff / se_paired_diff)
    
    norm = colors.Normalize(vmin=0, vmax=vmax, clip=True)
    discrete_cmap = plt.get_cmap('viridis', num_colors)

    fig, ax = plt.subplots(figsize=(9,3.5))
    img = ax.imshow(t_scores, cmap=discrete_cmap, norm=norm, origin='lower')
    fig.colorbar(img, ax=ax, orientation='vertical')
    ax.set_title("Paired difference t-scores")
    
    plt.tight_layout()
    if save_as:
        plt.savefig(save_as)
    plt.show()

In [ ]:
def plot_extracts(data, title, save_as=None):
    fig, ax = plt.subplots(1, 2, figsize=(9,3.5))
    
    to_plot = list(zip(np.median(data[::10,:,:,0],axis=2), np.arange(data.shape[0])[::10]))
    
    ax[0].set_title(title)
    for row, label in to_plot:
        ax[0].plot(row, label=label)
    
    ax[1].set_title(title + ' log-scale')
    for row, label in to_plot:
        ax[1].plot(row, label=label)
#     if log:
    ax[1].set_yscale('log')
    
    plt.legend(loc=0)
    plt.tight_layout()
    if save_as:
        plt.savefig(save_as)
    plt.show()

In [ ]:
#generic flow
Case = namedtuple('Case', 'name ndim vmin vmax max_diff')

cases = [
    Case('forrester', 1,  None,    None,  100),
    Case('forrester', 2,  None,    None,  100),
    Case('forrester', 4,  None,    None,   10),
    Case('branin',    2, .0001,   1_000,    1),
    Case('currin',    2,   .01,      10,   50),
    Case('park91a',   4,  None,    None,  100),
    Case('hartmann6', 6,  None,    None, 1e15),
    Case('borehole',  8, 1_000,   3_000,   10),
]

for c in cases:
    print(c.name, c.ndim)
    lin_mse_tracking = np.load(f'{data_dir}Matern_{c.ndim}d_{c.name}_lin_mse_tracking.npy')
#     print('median')
#     pprint([(f'{95+i}%-ile', np.nanpercentile(np.nanmedian(lin_mse_tracking, axis=2).flatten(), 95+i)) for i in range(6)])
    plot_name = f'{c.ndim}d-{c.name}-high-low-samples-linear'
    
    title = f'{c.name} ({c.ndim}D)'
    
#     plot_high_vs_low_num_samples(lin_mse_tracking, title, vmin=c.vmin, vmax=c.vmax, save_as=f'{plot_dir}{plot_name}.pdf')
#     plot_high_vs_low_num_samples_diff(lin_mse_tracking, title, max_diff=c.max_diff, save_as=f'{plot_dir}{plot_name}_diff.pdf')
#     display_paired_differences(lin_mse_tracking, save_as=f'{plot_dir}{plot_name}_significance.pdf')
    plot_extracts(lin_mse_tracking, title, save_as=f'{plot_dir}{plot_name}_extracts.pdf')

In [ ]:
lin_mse_tracking.shape

### Forrester function

In [ ]:
if 'Matern_1d_forrester_lin_mse_tracking.npy' in data_dir.iterdir:
    lin_mse_tracking = np.load(f'{data_dir}Matern_1d_forrester_lin_mse_tracking.npy')
else:
    pass
#     lin_mse_tracking = create_mse_tracking(TD_inv, low_lhs_sample)
#     np.save(f'{data_dir}Matern_1d_forrester_lin_mse_tracking.npy', lin_mse_tracking)

In [ ]:
print('median')
pprint([(f'{95+i}%-ile', np.nanpercentile(np.nanmedian(lin_mse_tracking, axis=2).flatten(), 95+i)) for i in range(6)])

In [ ]:
name = '2d-high-low-samples-linear'
plot_high_vs_low_num_samples(lin_mse_tracking, name, vmin=100, vmax=100_000, save_as=f'{plot_dir}{name}.pdf')

In [ ]:
name = '2d-high-low-samples-linear'
plot_high_vs_low_num_samples_diff(lin_mse_tracking, name, max_diff=100, save_as=f'{plot_dir}{name}_diff.pdf')

In [ ]:
paired_differences = lin_mse_tracking[:,:,:,1] - lin_mse_tracking[:,:,:,0]

In [ ]:
mean_paired_diff = np.mean(paired_differences, axis=2)
std_paired_diff = np.std(paired_differences, axis=2, ddof=1)
se_paired_diff = std_paired_diff / np.sqrt(lin_mse_tracking.shape[2])
t_scores = np.abs(mean_paired_diff / se_paired_diff)

In [ ]:
norm = colors.Normalize(vmin=0, vmax=5, clip=True)

discrete_cmap = plt.get_cmap('viridis', 5)

fig, ax = plt.subplots(figsize=(9,3.5))
img = ax.imshow(t_scores, cmap=discrete_cmap, norm=norm, origin='lower')
fig.colorbar(img, ax=ax, orientation='vertical')
ax.set_title("Paired difference t-scores")

In [ ]:
display_paired_differences(lin_mse_tracking)

fidelities: `high_hierarchical, high, low`

In [ ]:
lin_mse_tracking.shape

In [ ]:
paired_differences = lin_mse_tracking[:,:,:,1] - lin_mse_tracking[:,:,:,0]

In [ ]:
mean_paired_diff = np.mean(paired_differences, axis=2)
std_paired_diff = np.std(paired_differences, axis=2, ddof=1)
se_paired_diff = std_paired_diff / np.sqrt(lin_mse_tracking.shape[2])
t_scores = np.abs(mean_paired_diff / se_paired_diff)

In [ ]:
norm = colors.Normalize(vmin=0, vmax=5, clip=True)

discrete_cmap = plt.get_cmap('viridis', 5)

fig, ax = plt.subplots(figsize=(12,5))
img = ax.imshow(t_scores, cmap=discrete_cmap, norm=norm, origin='lower')
fig.colorbar(img, ax=ax, orientation='vertical')
ax.set_title("Paired difference t-scores")

As a function:

In [ ]:
def display_paired_differences(data, vmax=5, num_colors=5):
    paired_differences = data[:,:,:,0] - data[:,:,:,1]
    mean_paired_diff = np.mean(paired_differences, axis=2)
    std_paired_diff = np.std(paired_differences, axis=2, ddof=1)
    se_paired_diff = std_paired_diff / np.sqrt(data.shape[2])
    t_scores = abs(mean_paired_diff / se_paired_diff)
    
    norm = colors.Normalize(vmin=0, vmax=vmax, clip=True)
    discrete_cmap = plt.get_cmap('viridis', num_colors)

    fig, ax = plt.subplots(figsize=(12,5))
    img = ax.imshow(t_scores, cmap=discrete_cmap, norm=norm, origin='lower')
    fig.colorbar(img, ax=ax, orientation='vertical')
    ax.set_title("Paired difference t-scores")

In [ ]:
display_paired_differences(lin_mse_tracking)

### Branin function

In [ ]:
if 'Matern_2d_branin_lin_mse_tracking.npy' in data_dir.iterdir:
    branin_lin_mse_tracking = np.load(f'{data_dir}Matern_2d_branin_lin_mse_tracking.npy')
else:
    branin_lin_mse_tracking = create_mse_tracking(TD_inv, low_lhs_sample)
    np.save(f'{data_dir}2d_lin_mse_tracking.npy', lin_mse_tracking)

In [ ]:
print('median')
pprint([(f'{95+i}%-ile', np.nanpercentile(np.nanmedian(branin_lin_mse_tracking, axis=2).flatten(), 95+i)) for i in range(6)])

In [ ]:
name = 'Matern-2d-branin-high-low-samples-linear'
plot_high_vs_low_num_samples(branin_lin_mse_tracking, name, vmin=.0001, vmax=1_000, save_as=f'{plot_dir}{name}.pdf')

In [ ]:
name = 'Matern-2d-branin-high-low-samples-linear'
plot_high_vs_low_num_samples_diff(branin_lin_mse_tracking, name, max_diff=10, save_as=f'{plot_dir}{name}_diff.pdf')

In [ ]:
display_paired_differences(branin_lin_mse_tracking)

### Currin Function

In [ ]:
if 'Matern_2d_currin_lin_mse_tracking.npy' in data_dir.iterdir:
    currin_lin_mse_tracking = np.load(f'{data_dir}Matern_2d_currin_lin_mse_tracking.npy')
else:
    currin_lin_mse_tracking = create_mse_tracking(TD_inv, low_lhs_sample)
    np.save(f'{data_dir}2d_lin_mse_tracking.npy', lin_mse_tracking)

In [ ]:
print('median')
pprint([(f'{95+i}%-ile', np.nanpercentile(np.nanmedian(currin_lin_mse_tracking, axis=2).flatten(), 95+i)) for i in range(6)])

In [ ]:
name = 'Matern-2d-currin-high-low-samples-linear'
plot_high_vs_low_num_samples(currin_lin_mse_tracking, name, vmin=.01, vmax=10, save_as=f'{plot_dir}{name}.pdf')

In [ ]:
name = 'Matern-2d-currin-high-low-samples-linear'
plot_high_vs_low_num_samples_diff(currin_lin_mse_tracking, name, max_diff=50, save_as=f'{plot_dir}{name}_diff.pdf')

In [ ]:
display_paired_differences(currin_lin_mse_tracking)

### Borehole Function

In [ ]:
if 'Matern_8d_borehole_lin_mse_tracking.npy' in data_dir.iterdir:
    borehole_lin_mse_tracking = np.load(f'{data_dir}Matern_8d_borehole_lin_mse_tracking.npy')
else:
    borehole_lin_mse_tracking = create_mse_tracking(TD_inv, low_lhs_sample)
    np.save(f'{data_dir}2d_lin_mse_tracking.npy', lin_mse_tracking)

In [ ]:
print('median')
pprint([(f'{95+i}%-ile', np.nanpercentile(np.nanmedian(borehole_lin_mse_tracking, axis=2).flatten(), 95+i)) for i in range(6)])

In [ ]:
name = 'Matern-2d-borehole-high-low-samples-linear'
plot_high_vs_low_num_samples(borehole_lin_mse_tracking, name, vmin=100, vmax=10_000, save_as=f'{plot_dir}{name}.pdf')

In [ ]:
name = 'Matern-2d-borehole-high-low-samples-linear'
plot_high_vs_low_num_samples_diff(borehole_lin_mse_tracking, name, max_diff=10, save_as=f'{plot_dir}{name}_diff.pdf')

In [ ]:
display_paired_differences(borehole_lin_mse_tracking)

## Difference in error between linear and random sample

In [ ]:
name = "2D, random - LHS"
plot_inter_method_diff(mse_tracking, lin_mse_tracking, name, save_as=f'{plot_dir}{name}.pdf')

## Maximum found error per experiment

In [ ]:
errors = np.load(f'{data_dir}2d_error_tracking.npy')
max_errors = np.max(errors, axis=(2,4))

In [ ]:
name = '2d_max_error'

norm = colors.LogNorm(vmin=1, vmax=np.max(max_errors), clip=True)
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(20,4))
plt.suptitle(name)

for i, ax in enumerate(axes):
    img = ax.imshow(max_errors[:,:,i], cmap='viridis', norm=norm)
    
fig.colorbar(img, ax=axes[-1], orientation='vertical')
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_max_error.pdf')

In [ ]:
bins = [0] + [10**e for e in range(-3, 15)]
for i, m in enumerate(['hierarchical', 'high-only', 'low-only']):
    plt.hist(max_errors[:,:,i].flatten(), alpha=.3, bins=bins, label=m)
plt.xscale('log')
plt.legend()
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_max_error.pdf')

In [ ]:
lin_errors = np.load(f'{data_dir}2d_lin_error_tracking.npy')
max_lin_errors = np.max(lin_errors, axis=(2,4))

In [ ]:
name = '2d_lin_max_error'

norm = colors.LogNorm(vmin=1, vmax=np.max(max_lin_errors), clip=True)
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(20,4))
plt.suptitle(name)

for i, ax in enumerate(axes):
    img = ax.imshow(max_lin_errors[:,:,i], cmap='viridis', norm=norm)
    
fig.colorbar(img, ax=axes[-1], orientation='vertical')
plt.tight_layout()
plt.savefig(f'{plot_dir}2d_max_lin_error.pdf')

In [ ]:
bins = [0] + [10**e for e in range(-3, 15)]
for i, m in enumerate(['hierarchical', 'high-only', 'low-only']):
    plt.hist(max_lin_errors[:,:,i].flatten(), alpha=.3, bins=bins, label=m)
plt.xscale('log')
plt.legend()